In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida

import ipywidgets as ipw
import aiidalab_widgets_base as awb

from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

from aiidalab_widgets_base import WizardAppWidgetStep
from empa_molecules import steps, widgets

In [ ]:
def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)

        # Structure:
        with select_structure_step.manager.hold_sync():
            with select_structure_step.hold_sync():
                select_structure_step.confirmed_structure = process.inputs.structure
                select_structure_step.manager.input_structure = process.inputs.structure
        # Config.
        configure_calculation_step.dft_functional.value = process.inputs.functional
        configure_calculation_step.empirical_dispersion.value = process.inputs.empirical_dispersion
        configure_calculation_step.basis_set_opt.value = process.inputs.basis_set_opt
        configure_calculation_step.basis_set_scf.value = process.inputs.basis_set_scf
        configure_calculation_step.multiplicity_list.value = " ".join(map(str, process.inputs.multiplicity_list))
        configure_calculation_step.state = WizardAppWidgetStep.State.SUCCESS
        configure_calculation_step.confirm_button.disabled = True

        # Codes & resources
        submit_calculation_step.gaussian_code_dropdown.value = process.inputs.gaussian_code
        submit_calculation_step.formchk_code_dropdown.value = process.inputs.formchk_code
        submit_calculation_step.cubegen_code_dropdown.value = process.inputs.cubegen_code

        if "options" in process.inputs:
            submit_calculation_step.n_mpi_tasks_widget.value = process.inputs.options["resources"]["tot_num_mpiprocs"]
            submit_calculation_step.memory_widget.value = process.inputs.options["max_memory_kb"] / 1024 / 1.25
            submit_calculation_step.run_time_widget.value = process.inputs.options["max_wallclock_seconds"] / 60
        submit_calculation_step.state = WizardAppWidgetStep.State.SUCCESS

        # Running process.
        submit_calculation_step.process = process


In [ ]:
# Add process selection header
work_chain_selector = widgets.WorkChainSelectorWidget(layout=ipw.Layout(width='auto'))
work_chain_selector.observe(_observe_process_selection, 'value')    

select_structure_step = steps.StructureSelectionStep(auto_advance=True)

configure_calculation_step = steps.ConfigureGaussianCalculationStep(auto_advance=True)

ipw.dlink((select_structure_step, 'confirmed_structure'), (configure_calculation_step, 'input_structure'))


submit_calculation_step = steps.SubmitGaussianCalculationStep(auto_advance=True)
ipw.dlink((configure_calculation_step, 'inputs'), (submit_calculation_step, "inputs"))


results = steps.ViewGaussianWorkChainStatusAndResultsStep(auto_advance=True)
ipw.dlink((submit_calculation_step, 'process'), (results, 'process'))        
ipw.dlink((submit_calculation_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)

app = awb.WizardAppWidget(
    steps=[
        ("Select structure", select_structure_step),
        ("Cofigure calculation", configure_calculation_step),
        ("Submit", submit_calculation_step),
        ("Results", results)
    ]
)

app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

output = ipw.Output()
install_create_github_issue_exception_handler(
    output,
    url='https://github.com/nanotech-empa/aiidalab-empa-molecules/issues/new',
    labels=('bug', 'automated-report'))

with output:
    display(app_with_work_chain_selector)
    
display(output)